In [21]:
 import warnings
 warnings.filterwarnings('ignore')

In [22]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
file = "2010 Federal STEM Education Inventory Data Set.xls"
x1=pd.ExcelFile(file)
df2=x1.parse(0,skiprows=[0],nrows= 252,index_col="Index Number")
#df2.info(verbose=True)

In [5]:
df2.drop(["B) Year Established"], axis=1,inplace=True)

In [6]:
LABELS=["C1) Funding FY2008","C2) Funding FY2009","C3) Funding FY2010"]

In [7]:
import sklearn
from sklearn.preprocessing  import Imputer
imp = Imputer(missing_values="NaN",strategy="median",axis=0)
imp.fit(df2[LABELS])
df2[LABELS]=imp.transform(df2[LABELS])

In [8]:
df2['+ve%growth'] = np.where(((df2["C2) Funding FY2009"]-df2["C1) Funding FY2008"])/df2["C1) Funding FY2008"]*100>0), 1, 0)
df2.head(10)

,Investment Name,Agency,Subagency,A) Brief Description,C1) Funding FY2008,C2) Funding FY2009,C3) Funding FY2010,D) Mission-specific or General STEM?,E) Agency or Mission-Related Workforce Needs?,F1) Primary Investment Objective,...,Unnamed: 247,Y3) Process or Implementation Evaluation Type,Unnamed: 249,Unnamed: 250,Y4) Portfolio Evaluation/Review Type,Unnamed: 252,Unnamed: 253,Y5) Expert Review Type,Unnamed: 255,+ve%growth
Index Number,,,,,,,,,,,,,,,,,,,,,
PRG_0001,Centers for Ocean Sciences Education Excellence ~,National Science Foundation,Directorate for Geosciences (GEO),The Division of Ocean Sciences seeks to establ...,5.24,7.19,5.70,General STEM,NaN,Institutional Capacity: Support advancement an...,...,NaN,NaN,NaN,NO,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN,1
PRG_0002,Broadening Participation in Computing (BPC),National Science Foundation,Directorate for Computer & Information Science...,The BPC Program was not primarily aimed at edu...,14.00,14.00,14.00,General STEM,NaN,"Engagement: Increase learners’ engagement, int...",...,NaN,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,0
PRG_0003,CISE Pathways to Revitalized Undergraduate Com...,National Science Foundation,Directorate for Computer & Information Science...,Through the CISE Pathways to Revitalized Under...,5.00,5.00,4.37,General STEM,NaN,Institutional Capacity: Support advancement an...,...,NaN,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,0
PRG_0004,Opportunities for Enhancing Diversity in the G...,National Science Foundation,Directorate for Geosciences (GEO),The Opportunities for Enhancing Diversity in t...,4.57,11.79,4.18,General STEM,NaN,"Engagement: Increase learners’ engagement, int...",...,NO,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN,NaN,INDEPENDENT,NaN,1
PRG_0005,Geoscience Education,National Science Foundation,Directorate for Geosciences (GEO),The Geoscience Education (GeoEd) program is fo...,1.63,2.74,2.02,General STEM,NaN,"Learning: Develop STEM skills, practices, or k...",...,NO,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,1
PRG_0006,Geoscience Teacher Training (GEO-Teach),National Science Foundation,Directorate for Geosciences (GEO),The Geoscience Teacher Training (GEO-Teach) pr...,3.00,3.00,2.98,General STEM,NaN,Pre and In Service Educator/Education Leader P...,...,NO,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,0
PRG_0007,Global Learning and Observations to Benefit th...,National Science Foundation,Directorate for Geosciences (GEO),The GLOBE Program is a hands-on international ...,1.10,1.12,1.10,General STEM,NaN,"Engagement: Increase learners’ engagement, int...",...,NO,NaN,NaN,NO,NaN,NaN,NO,NaN,NaN,1
PRG_0008,Graduate Research Fellowship Program (GRFP),National Science Foundation,Directorate for Education and Human Resources ...,The purpose of the NSF Graduate Research Fello...,96.02,162.43,136.13,General STEM,NaN,Post-Secondary STEM Degrees: Increase the numb...,...,NO,NaN,NaN,NO,NaN,NaN,NO,INDEPENDENT,NaN,1
PRG_0009,Integrative Graduate Education and Research Tr...,National Science Foundation,Directorate for Education and Human Resources ...,IGERT is an NSF-wide program intended to meet ...,64.76,77.99,69.70,General STEM,NaN,Post-Secondary STEM Degrees: Increase the numb...,...,NO,NaN,NaN,NO,NaN,NaN,NO,INDEPENDENT,NaN,1


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,roc_curve
numeric_data_only=df2[LABELS]
X_train, X_test, y_train, y_test = train_test_split(df2[LABELS], df2['+ve%growth'], test_size = 0.30, random_state=42, stratify=df2['+ve%growth'])
clf=(LogisticRegression())
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
fpr,tpr,thresholds=roc_curve(y_test,y_pred)
print("ROC_AUC: {}".format(roc_auc_score(y_test,y_pred)))

ROC_AUC: 0.8548387096774194


In [12]:
df1=df2.loc[:, pd.notnull(df2).sum()>=len(df2)*.20]
#df1.info()

In [13]:
NUMERIC_COLUMNS =[ "C1) Funding FY2008","C2) Funding FY2009","C3) Funding FY2010","+ve%growth"]
NUMERIC_COLUMNS_T= [ "C1) Funding FY2008","C2) Funding FY2009","C3) Funding FY2010"]

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop(['+ve%growth'],axis=1), df1['+ve%growth'], test_size = 0.30, random_state=42, stratify=df1['+ve%growth'])

def combine_text_columns(data_frame, to_drop=NUMERIC_COLUMNS_T):
    """ converts all text in each row of data_frame to single vector """
    
    # Drop non-text columns that are in the df
    to_drop = set(to_drop) & set(data_frame.columns.tolist())
    text_data = data_frame.drop(to_drop,axis=1)
    
    # Replace nans with blanks
    text_data.fillna("",inplace=True)
    
    # Join all text items in a row that have a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)
# get text data
from sklearn.preprocessing import FunctionTransformer,MaxAbsScaler
get_text_data = FunctionTransformer(combine_text_columns,validate=False)

# Preprocess the numeric data: get_numeric_data
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC_COLUMNS_T], validate=False)



In [19]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import HashingVectorizer
chi_k=250
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ("selector", get_numeric_data),
            ])),
                ('text_features', Pipeline([
                    ("selector",get_text_data),
                    ("vectorizer", HashingVectorizer(token_pattern=TOKENS_ALPHANUMERIC,non_negative=True, norm=None, binary=False, ngram_range=(1,2))),
                    ('dim_red', SelectKBest(chi2, chi_k))
                ]))
             ])), ("clf",LogisticRegression())
        ])


In [23]:
#BEFORE HYPER PARAMETER TUNING
pl.fit(X_train,y_train)
y_pred=pl.predict(X_test)
print("ROC_AUC: {}".format(roc_auc_score(y_test,y_pred)))

ROC_AUC: 0.7347670250896057


In [24]:
#LIFT USING ONLY TEXT DATA in pl_1
pl_1 = Pipeline([("selector",get_text_data),
                    ("vectorizer",CountVectorizer()),("clf",LogisticRegression())
        ])


In [25]:
pl_1.fit(X_train,y_train)
y_pred=pl_1.predict(X_test)
print("ROC_AUC: {}".format(roc_auc_score(y_test,y_pred)))

ROC_AUC: 0.48351254480286743


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Create the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid = {"clf__C": c_space,"clf__penalty": ['l1', 'l2']}

# Instantiate the GridSearchCV object: logreg_cv
logreg_cv = GridSearchCV(pl,param_grid,cv=5)


In [27]:

logreg_cv.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('numeric_features', Pipeline(memory=None,
     steps=[('selector', FunctionTransformer(accept_sparse=False,
          func=<function <lambda> at 0x0000017A25D11488>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y=...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__C': array([1.00000e-05, 8.48343e-05, 7.19686e-04, 6.10540e-03, 5.17947e-02,
       4.39397e-01, 3.72759e+00, 3.16228e+01, 2.68270e+02, 2.27585e+03,
       1.93070e+04, 1.63789e+05, 1.38950e+06, 1.17877e+07, 1.00000e+08]), 'clf__penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
print("Tuned Logistic Regression: {}".format(logreg_cv.best_params_))

Tuned Logistic Regression: {'clf__C': 0.4393970560760795, 'clf__penalty': 'l1'}


In [29]:
y_pred = logreg_cv.predict(X_test)

In [30]:
#FINAL AUC AFTER GRID SEARCH
print("ROC_AUC: {}".format(roc_auc_score(y_test,y_pred)))

ROC_AUC: 0.7992831541218638
